In [57]:
import csv
import itertools
import operator
import numpy as np
import nltk
import sys
from datetime import datetime

import matplotlib.pyplot as plt
%matplotlib inline

In [58]:
vocabulary_size = 8000
unknown_token = "UNKNOWN_TOKEN"
sentence_start_token = "SENTENCE_START"
sentence_end_token = "SENTENCE_END"

In [59]:
with open('data/comments.csv', 'rb') as f:
    reader = csv.reader(f, skipinitialspace=True)
    reader.next()
    # Split full comments into sentences
    sentences = itertools.chain(*[nltk.sent_tokenize(x[0].decode('utf-8').lower()) for x in reader])
    sentences = ["%s %s %s" % (sentence_start_token, x, sentence_end_token) for x in sentences]

In [60]:
tokenized_sentences = [nltk.word_tokenize(sent) for sent in sentences]

In [61]:
word_freq = nltk.FreqDist(itertools.chain(*tokenized_sentences))

In [62]:
vocab = word_freq.most_common(vocabulary_size-1)
index_to_word = [x[0] for x in vocab]
index_to_word.append(unknown_token)
word_to_index = dict([(w,i) for i,w in enumerate(index_to_word)])

In [63]:
for i, sent in enumerate(tokenized_sentences):
    tokenized_sentences[i] = [w if w in word_to_index else unknown_token for w in sent]

In [64]:
train_X = np.asarray([[word_to_index[w] for w in sent[:-1]] for sent in tokenized_sentences])
train_y = np.asarray([[word_to_index[w] for w in sent[1:]] for sent in tokenized_sentences])

In [65]:
def softmax(x):
    exps = np.exp(x)
    return exps / np.sum(exps)

In [136]:
class RNNNumpy(object):
    def __init__(self, word_dim, hidden_dim=100, bptt_truncate=4):
        self.word_dim = word_dim
        self.hidden_dim = hidden_dim
        self.bptt_truncate = bptt_truncate
        self.U = np.random.uniform(-1./np.sqrt(word_dim), 1./np.sqrt(word_dim), (hidden_dim, word_dim))
        self.W = np.random.uniform(-1./np.sqrt(hidden_dim), 1./np.sqrt(hidden_dim), (hidden_dim, hidden_dim))
        self.V = np.random.uniform(-1./np.sqrt(hidden_dim), 1./np.sqrt(hidden_dim), (word_dim, hidden_dim))
        
    def forward(self, x):
        T = len(x)
        o = np.zeros((T, self.word_dim))
        s = np.zeros((T+1, self.hidden_dim))
        for t in range(T):
            s[t] = np.tanh(self.U[:,x[t]] + self.W.dot(s[t-1]))
            o[t] = softmax(self.V.dot(s[t]))
        return [o, s]
    
    def predict(self, x):
        o, s = self.forward(x)
        return np.argmax(o, axis=1)
    
    def calculate_total_loss(self, x, y):
        loss = 0.0
        for i in range(len(y)):
            o, s = self.forward(x[i])
            for j in range(len(y[i])):
                loss += -np.log(o[j, y[i][j]])
        return loss
    
    def calculate_loss(self, x, y):
        total_loss = self.calculate_total_loss(x, y)
        N = np.sum(len(y[i]) for i in range(len(y)))
        return total_loss / N
       
    def backward(self, x, y):
        T = len(y)
        o, s = self.forward(x)
        dLdU = np.zeros(self.U.shape)
        dLdW = np.zeros(self.W.shape)
        dLdV = np.zeros(self.V.shape)
        
        delta_o = o
        delta_o[np.arange(T), y] -= 1.
        for t in np.arange(T)[::-1]:
            dLdV += np.outer(delta_o[t], s[t].T)
            delta_t = self.V.T.dot(delta_o[t]) * (1 - (s[t]**2))
            for tt in np.arange(max(0, t-self.bptt_truncate), t+1)[::-1]:
                dLdW += np.outer(delta_t, s[tt-1])
                dLdU[:, x[tt]] += delta_t
                delta_t = self.W.T.dot(delta_t) * (1 - s[tt-1]**2)
        return [dLdU, dLdV, dLdW]
    
    def bptt(self, x, y):
        T = len(y)
        o, s = self.forward(x)
        dLdU = np.zeros(self.U.shape)
        dLdW = np.zeros(self.W.shape)
        dLdV = np.zeros(self.V.shape)
        
        delta_o = o
        delta_o[np.arange(T), y] -= 1.
        # For each output backwards...
        for t in np.arange(T)[::-1]:
            dLdV += np.outer(delta_o[t], s[t].T)
            # Initial delta calculation
            delta_t = self.V.T.dot(delta_o[t]) * (1 - (s[t]**2))
            for bptt_step in np.arange(max(0, t-self.bptt_truncate), t+1)[::-1]:
                # print "Backpropagation step t=%d bptt step=%d " % (t, bptt_step)
                dLdW += np.outer(delta_t, s[bptt_step-1])
                #dLdW += np.outer(delta_t, s[bptt_step-1])  
                dLdU[:, x[bptt_step]] += delta_t
                #dLdU[:,x[bptt_step]] += delta_t
                # Update delta for next step
                delta_t = self.W.T.dot(delta_t) * (1 - s[bptt_step-1]**2)
                #delta_t = self.W.T.dot(delta_t) * (1 - s[bptt_step-1] ** 2)
        return [dLdU, dLdV, dLdW]
    
    def gradient_check(self, x, y, h=0.001, error_threshold=0.01):
        gradients = self.bptt(x, y)
        params = ['U', 'V', 'W']
        for idx, name in enumerate(params):
            param = operator.attrgetter(name)(self)
            print "Performing gradient check for parameter %s with size %d." % (name, np.prod(param.shape))
            it = np.nditer(param, flags=['multi_index'], op_flags=['readwrite'])
            while not it.finished:
                i = it.multi_index
                value = param[i]
                param[i] = value - h
                minus = self.calculate_total_loss([x], [y])
                param[i] = value + h
                plus = self.calculate_total_loss([x], [y])
                param[i] = value
                approx = (plus - minus) / (2 * h)
                grad = gradients[idx][i]
                rel_error = np.abs(grad - approx) / (np.abs(grad) + np.abs(approx))  
                if i == (0, 0):
                    print approx, grad
                if rel_error > error_threshold:
                    print "Gradient Check ERROR: parameter=%s ix=%s" % (name, i)
                    print "+h Loss: %f" % plus
                    print "-h Loss: %f" % minus
                    print "Estimated_gradient: %f" % approx
                    print "Backpropagation gradient: %f" % grad
                    print "Relative Error: %f" % rel_error
                    return
                it.iternext()
            print "Gradient check for parameter %s passed." % (name)
        
            
        
        
        
            

In [137]:
grad_check_vocab_size = 100
np.random.seed(10)
model = RNNNumpy(grad_check_vocab_size, 10, bptt_truncate=1000)
model.gradient_check([0,1,2,3], [1,2,3,4])

Performing gradient check for parameter U with size 1000.
0.0254028668767 0.0254028829584


/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:96: RuntimeWarning: invalid value encountered in double_scalars


Gradient check for parameter U passed.
Performing gradient check for parameter V with size 1000.
0.00142997246577 0.0014299724647
Gradient check for parameter V passed.
Performing gradient check for parameter W with size 100.
0.0230553273237 0.0230553177573
Gradient check for parameter W passed.


In [108]:
rnn.gradient_check([1,2, 3], [2,3,4])

Performing gradient check for parameter U with size 100.
Gradient check for parameter U passed.
Performing gradient check for parameter V with size 100.
Gradient Check ERROR: parameter=V ix=(0, 0)
+h Loss: 7.032527
-h Loss: 7.032449
Estimated_gradient: 0.039091
Backpropagation gradient: -0.178315
Relative Error: 1.000000


/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:71: RuntimeWarning: invalid value encountered in double_scalars


In [82]:
train_y[0]

[6,
 3513,
 7,
 155,
 794,
 25,
 223,
 8,
 32,
 20,
 202,
 5025,
 350,
 91,
 6,
 66,
 207,
 5,
 2,
 1]